Setup

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.parent.absolute()))
import config
import argparse
from pprint import pprint
from time import sleep
from context import Context
from contracts.farm_contract import FarmContract
from contracts.staking_contract import StakingContract
from contracts.metastaking_contract import MetaStakingContract
from contracts.unstaker_contract import UnstakerContract
from utils.utils_generic import get_file_from_url_or_path, get_continue_confirmation
from utils.utils_chain import get_bytecode_codehash, base64_to_hex, WrapperAddress
from tools.runners.farm_runner import upgrade_farmv2_contract
from tools.runners.staking_runner import upgrade_staking_contracts
from tools.runners.metastaking_runner import upgrade_metastaking_contract
from tools.runners.generic_runner import upgrade_generic_contract

context = Context()

In [ ]:
farm_contract = FarmContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqr9gx79xls33tnzt0a94uvnkf452fz26q2jpsau4ulw")
staking_contract = StakingContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqqsx29p3fge7upkgup4mm5xsdsv4w7rh82jpsdcvdrt")
metastaking_contract = MetaStakingContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqxjdlk9d8gap8q2ghey6q5cg69y7p4rkd2jpsv89u3n")
unstaker_contract: UnstakerContract = context.get_contracts(config.UNSTAKERS)[0]

print("Farm contract:")
pprint(farm_contract.get_config_dict())
print("Staking contract:")
pprint(staking_contract.get_config_dict())
print("MetaStaking contract:")
pprint(metastaking_contract.get_config_dict())
print("Unstaker contract:")
pprint(unstaker_contract.get_config_dict())

Farm upgrade

In [ ]:
farm_bytecode_path = ""
expected_code_hash = ""

bytecode_path = get_file_from_url_or_path(farm_bytecode_path)
bytecode_codehash = get_bytecode_codehash(bytecode_path)
print(f"New bytecode codehash: {bytecode_codehash}")
assert bytecode_codehash == expected_code_hash

if not get_continue_confirmation(config.FORCE_CONTINUE_PROMPT):
    raise Exception("User cancelled")

tx_hash = farm_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, bytecode_path, [], True)

context.network_provider.check_simple_tx_status(tx_hash, f"upgrade farm contract: {farm_contract.address}")

code_hash = context.network_provider.proxy.get_account(WrapperAddress(farm_contract.address)).code_hash
assert base64_to_hex(code_hash) == expected_code_hash

In [ ]:
farm_bytecode_path = ""
expected_code_hash = ""

args = argparse.Namespace(address=farm_contract.address, bytecode=farm_bytecode_path, compare_states=True, all=False)
upgrade_farmv2_contract(args)

code_hash = context.network_provider.proxy.get_account(WrapperAddress(farm_contract.address)).code_hash
assert base64_to_hex(code_hash) == expected_code_hash

Staking upgrade

In [ ]:
staking_bytecode_path = ""
expected_code_hash = ""

bytecode_path = get_file_from_url_or_path(staking_bytecode_path)
bytecode_codehash = get_bytecode_codehash(bytecode_path)
print(f"New bytecode codehash: {bytecode_codehash}")
assert bytecode_codehash == expected_code_hash

if not get_continue_confirmation(config.FORCE_CONTINUE_PROMPT):
    raise Exception("User cancelled")

tx_hash = staking_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, bytecode_path, [], True)

context.network_provider.check_simple_tx_status(tx_hash, f"upgrade staking contract: {staking_contract.address}")

code_hash = context.network_provider.proxy.get_account(WrapperAddress(staking_contract.address)).code_hash
assert base64_to_hex(code_hash) == expected_code_hash

In [ ]:
staking_bytecode_path = ""
expected_code_hash = ""

args = argparse.Namespace(address=staking_contract.address, bytecode=staking_bytecode_path, compare_states=True, all=False)
upgrade_staking_contracts(args)

code_hash = context.network_provider.proxy.get_account(WrapperAddress(staking_contract.address)).code_hash
assert base64_to_hex(code_hash) == expected_code_hash

Metastaking upgrade

In [ ]:
metastaking_bytecode_path = ""
expected_code_hash = ""

args = argparse.Namespace(address=metastaking_contract.address, bytecode=metastaking_bytecode_path, compare_states=True, all=False)
upgrade_metastaking_contract(args)

code_hash = context.network_provider.proxy.get_account(WrapperAddress(metastaking_contract.address)).code_hash
assert base64_to_hex(code_hash) == expected_code_hash

Unstaker upgrade

In [ ]:
unstaker_bytecode_path = ""
expected_code_hash = ""

args = argparse.Namespace(address=unstaker_contract.address, bytecode=unstaker_bytecode_path, compare_states=True, all=False)
upgrade_generic_contract(args)

code_hash = context.network_provider.proxy.get_account(WrapperAddress(unstaker_contract.address)).code_hash
assert base64_to_hex(code_hash) == expected_code_hash

Farm resume

In [ ]:
tx_hash = farm_contract.resume(context.deployer_account, context.network_provider.proxy)

Staking resume

In [ ]:
tx_hash = staking_contract.resume(context.deployer_account, context.network_provider.proxy)